In [1]:
import os

os.chdir("..")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import tqdm.auto as tqdm

from datasets import load_dataset

from imperative_stitch.to_s import (
    pair_to_s_exp,
    python_to_s_exp,
    s_exp_to_python,
    s_exp_parse,
)
from imperative_stitch.compress.julia_stitch import run_julia_stitch

from s_expression_parser import nil

In [4]:
STITCH_JL_DIR = "../Stitch.jl"

In [5]:
tests = ["(+ x 2)", "(+ x 3)"]

In [6]:
dset = load_dataset("deepmind/code_contests", split="train")

Found cached dataset parquet (/home/kavi/.cache/huggingface/datasets/deepmind___parquet/deepmind--code_contests-c83aff5e5b92def4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [7]:
sets = []
for datum in dset:
    py3s = [
        sol
        for sol, lang in zip(
            datum["solutions"]["solution"], datum["solutions"]["language"]
        )
        if lang == 3
    ]
    if len(py3s) > 3:
        sets.append(py3s)
    if len(sets) > 5:
        break

In [8]:
codes = sets[0][7:1+8]

for i, x in enumerate(codes):
    print("*" * 10 + " " * 10 + str(i) + " " * 10 + "*" * 10)
    print(x)

**********          0          **********
n=int(input())
s=input()
a=s.count('8')
b=n//11
print(min(a,b))


**********          1          **********
n=int(input())
s=input()
c=s.count('8')
x=n//11
if c>=x:
    print(n//11)
elif c < x:
    print(c)
else:
    print(0)


In [9]:
x = """def min_iterations_to_sort(n, a):
    is_sorted = False
    iterations = 0

    while not is_sorted:
        is_sorted = True
        iterations += 1

        if iterations % 2 == 1:
            for i in range(0, n - 1, 2):
                if a[i] > a[i + 1]:
                    a[i], a[i + 1] = a[i + 1], a[i]
                    is_sorted = False
        else:
            for i in range(1, n - 1, 2):
                if a[i] > a[i + 1]:
                    a[i], a[i + 1] = a[i + 1], a[i]
                    is_sorted = False

    return iterations"""
y = """def min_iterations_to_sort(n, a):
    sorted_a = sorted(a)
    odd_swaps = 0
    even_swaps = 0
    
    for i in range(n):
        if i % 2 == 0:
            for j in range(0, n - 1, 2):
                if a[j] > a[j + 1]:
                    a[j], a[j + 1] = a[j + 1], a[j]
                    odd_swaps += 1
        else:
            for j in range(1, n - 1, 2):
                if a[j] > a[j + 1]:
                    a[j], a[j + 1] = a[j + 1], a[j]
                    even_swaps += 1
    
    if a == sorted_a:
        return max(odd_swaps, even_swaps)
    else:
        return -1"""

In [10]:
tests = [python_to_s_exp(code) for code in codes]
tests = [python_to_s_exp(code) for code in ["x = 2\nx=3\nx=4", "y=2\ny=3\ny=4"]]
tests = [python_to_s_exp(code) for code in [x, y]]

In [11]:
_, abstractions, rewritten = run_julia_stitch(
    tests,
    stitch_jl_dir="../Stitch.jl/",
    iters=1,
    max_arity=2,
    quiet=False,
)

Run the following command to debug:
julia --project=../Stitch.jl/ ../Stitch.jl/src/cli.jl --iterations=1 --max-arity=2 --dfa=/home/kavi/mit/pystitch/data/dfa.json < ../Stitch.jl/temp.txt
===Iteration 1===
Best abstraction: (Assign (list (Tuple (list (Subscript (Name %2 Load) (Name %1 Load) Store) (Subscript (Name %2 Load) (BinOp (Name %1 Load) Add (Constant i1 None)) Store)) Store)) (Tuple (list (Subscript (Name %2 Load) (BinOp (Name %1 Load) Add (Constant i1 None)) Load) (Subscript (Name %2 Load) (Name %1 Load) Load)) Load) None) with utility 141.0 compressed by 1.25x
Stitch.Stats(
    expansions=294
    completed=11
    comparable_worklist_steps=250
    )
Total compression: 1.374269x
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[{"body":"(Assign (list (Tuple (list (Subscript (Name %2 Load) (Name %1 Load) Store) (Subscript (Name %2 Load) (BinOp (Name %1 Load) Add (Constant i1 None)) Store)) Store)) (Tuple (list (Subscript (Name %2 Load) (BinOp (Name %1 Load) Add (Constant i1 None)) Load) (Subscrip

In [12]:
abstractions

[{'body': '(Assign (list (Tuple (list (Subscript (Name %2 Load) (Name %1 Load) Store) (Subscript (Name %2 Load) (BinOp (Name %1 Load) Add (Constant i1 None)) Store)) Store)) (Tuple (list (Subscript (Name %2 Load) (BinOp (Name %1 Load) Add (Constant i1 None)) Load) (Subscript (Name %2 Load) (Name %1 Load) Load)) Load) None)',
  'sym_arity': 2,
  'arity': 0}]

In [13]:
for abstraction in abstractions:
    print(s_exp_to_python(abstraction["body"]))

(%2[%1], %2[%1 + 1]) = (%2[%1 + 1], %2[%1])


In [14]:
print(s_exp_to_python(rewritten[0]))

def min_iterations_to_sort(n, a):
    is_sorted = False
    iterations = 0
    while not is_sorted:
        is_sorted = True
        iterations += 1
        if iterations % 2 == 1:
            for i in range(0, n - 1, 2):
                if a[i] > a[i + 1]:fn_1(i, a)
                    is_sorted = False
        else:
            for i in range(1, n - 1, 2):
                if a[i] > a[i + 1]:fn_1(i, a)
                    is_sorted = False
    return iterations
